# DX 704 Week 10 Project

In this project, you will implement document search within a question and answer database and assess its performance.


The full project description and a template notebook are available on GitHub: [Project 10 Materials](https://github.com/bu-cds-dx704/dx704-project-10).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download the SQuAD-explorer Data Set

You may use the code provided below.

In [1]:
!git clone https://github.com/rajpurkar/SQuAD-explorer

Cloning into 'SQuAD-explorer'...
remote: Enumerating objects: 5563, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 5563 (delta 11), reused 17 (delta 6), pack-reused 5539 (from 1)
Receiving objects: 100% (5563/5563), 52.26 MiB | 418.00 KiB/s, done.
Resolving deltas: 100% (3563/3563), done.


In [2]:
import json

In [3]:
with open("SQuAD-explorer/dataset/train-v1.1.json") as fp:
    train_data = json.load(fp)

In [ ]:
type(train_data)

dict

In [ ]:
list(train_data.keys())

['data', 'version']

In [ ]:
type(train_data["data"])

list

In [ ]:
len(train_data["data"])

442

In [ ]:
type(train_data["data"][0])

dict

In [ ]:
train_data["data"][0].keys()

dict_keys(['title', 'paragraphs'])

In [ ]:
train_data["data"][0]["title"]

'University_of_Notre_Dame'

In [ ]:
len(train_data["data"][0]["paragraphs"])

55

In [ ]:
train_data["data"][0]["paragraphs"][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [ ]:
sum(len(doc["paragraphs"]) for doc in train_data["data"])

18896

## Part 2: Restructure JSON Data for Processing

Parse the file "SQuAD-explorer/dataset/train-v1.1.json" above to produce a file "parsed.tsv" with columns document_title, paragraph_index, and paragraph_context.
The paragraph_index column should be zero-indexed, so zero for the first paragraph of each document.
Use pandas `to_csv` method to write the file since there are many quotes and other issues to handle otherwise.

In [4]:
# YOUR CHANGES HERE
import pandas as pd

rows = []
for article in train_data["data"]:
    title = article["title"]
    for i, para in enumerate(article["paragraphs"]):
        context = para["context"]
        rows.append({
            "document_title": title,
            "paragraph_index": i,
            "paragraph_context": context
        })

df = pd.DataFrame(rows, columns=["document_title", "paragraph_index", "paragraph_context"])

df.to_csv("parsed.tsv", sep="\t", index=False)

Submit "parsed.tsv" in Gradescope.

## Part 3: Prepare Suitable Paragraph Vectors for Document Search

Design and implement paragraph vectors based on their text with length 1024.
Note that this will be much smaller than the number of distinct words in the training data.

Hint: you can base your vectors on any techniques covered in this module so far.
Beware that they will be automatically assessed (along with the question vectors of part 4) to make sure they retain useful information.

In [5]:
# YOUR CHANGES HERE

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np

df = pd.read_csv("parsed.tsv", sep="\t")

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["paragraph_context"])

print("TF-IDF matrix shape:", tfidf_matrix.shape)  

svd = TruncatedSVD(n_components=1024, random_state=42)
paragraph_vectors = svd.fit_transform(tfidf_matrix)

print("Final embedding shape:", paragraph_vectors.shape)  

df["paragraph_vector_json"] = [
    json.dumps(vec.tolist()) for vec in paragraph_vectors
]

df[["document_title", "paragraph_index", "paragraph_vector_json"]].to_csv(
    "paragraph-vectors.tsv.gz",
    sep="\t",
    index=False,
    compression="gzip"
)

TF-IDF matrix shape: (18896, 78537)
Final embedding shape: (18896, 1024)


Save your paragraph vectors in a file "paragraph-vectors.tsv.gz" with columns document_title, paragraph_index, and paragraph_vector_json where paragraph_vector_json is a JSON encoded list.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

Submit "paragraph-vectors.tsv.gz" in Gradescope.

## Part 4: Encode Question Vectors with the Same Design

Read the questions in "questions.tsv" and encode them in the same way that you encoded the paragraph vectors.

In [6]:
# YOUR CHANGES HERE

questions = pd.read_csv("questions.tsv", sep="\t")

q_tfidf = vectorizer.transform(questions["question"])

q_vecs = svd.transform(q_tfidf)

questions["question_vector_json"] = [
    json.dumps(vec.tolist()) for vec in q_vecs
]

questions[["question_id", "question_vector_json"]].to_csv(
    "question-vectors.tsv",
    sep="\t",
    index=False
)

Save your question vectors in "question-vectors.tsv" with columns question_id and question_vector_json.

Submit "question-vectors.tsv" in Gradescope.

## Part 5: Match Questions to Paragraphs using Nearest Neighbors

Match your question vectors to paragraph vectors and identify the top 5 paragraph vectors for each question using nearest neighbors.
Specifically, use the Euclidean distance between the vectors.


In [7]:
# YOUR CHANGES HERE

from sklearn.neighbors import NearestNeighbors

qdf = pd.read_csv("question-vectors.tsv", sep="\t")

pdf = pd.read_csv("paragraph-vectors.tsv.gz", sep="\t", compression="gzip")

q_mat = np.vstack(qdf["question_vector_json"].apply(lambda x: np.array(json.loads(x))))
p_mat = np.vstack(pdf["paragraph_vector_json"].apply(lambda x: np.array(json.loads(x))))

nn = NearestNeighbors(n_neighbors=5, metric="euclidean")
nn.fit(p_mat)

distances, indices = nn.kneighbors(q_mat, return_distance=True)

rows = []
for qi, qid in enumerate(qdf["question_id"]):
    for rank, pidx in enumerate(indices[qi], start=1):
        rows.append({
            "question_id": qid,
            "question_rank": rank,
            "document_title": pdf.iloc[pidx]["document_title"],
            "paragraph_index": int(pdf.iloc[pidx]["paragraph_index"])
        })

match_df = pd.DataFrame(rows, columns=["question_id", "question_rank", "document_title", "paragraph_index"])
match_df.to_csv("question-matches.tsv", sep="\t", index=False)


Save your top matches in a file "question-matches.tsv" with columns question_id, question_rank, document_title, and paragraph_index.


Submit "question-matches.tsv" in Gradescope.

## Part 6: Spot Check Question and Paragraph Matches

Review the paragraphs matched to the first 5 questions (sorted by question_id ascending).
Which paragraph was the worst match for each question?


Submit "worst-paragraphs.tsv" in Gradescope.

Write a file "worst-paragraphs.tsv" with three columns question_id, document_title, paragraph_index.

In [8]:
matches = pd.read_csv("question-matches.tsv", sep="\t")

first_five_qids = sorted(matches["question_id"].unique())[:5]

subset = matches[matches["question_id"].isin(first_five_qids)]

worst = subset[subset["question_rank"] == 5][
    ["question_id", "document_title", "paragraph_index"]
]

worst.to_csv("worst-paragraphs.tsv", sep="\t", index=False)

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 8: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

None. 